# Observer design pattern

**Used for updating multiple subscribers of any changes that happen in the publisher.**

Write a class Welcome that updates the mailing list, a telegram chat, and a slack chat of the creation of new users in a company.

It has to work with the following main.


In [4]:
class CompanyUpdate:
    def __init__(self, events):
        self.subscribers = {event: dict() for event in events}
    
    def get_subscribers(self, event):
        if event in self.subscribers:
            return self.subscribers[event]
        else:
            raise ValueError

    def register(self, event, subscriber, callback_fn=None):
        default_method = 'update'

        if callback_fn is None:
            try:
                callback_fn = getattr(subscriber, default_method)
            except:
                raise ValueError("Impossible to subscribe. The subscriber should provide "
                                 "an update method or pass a callback.")
        self.get_subscribers(event)[subscriber] = callback_fn

    def unregister(self, event, subscriber):
        subscribers_for_event = self.get_subscribers(event)
        if subscriber in subscribers_for_event:
            del self.get_subscribers(event)[subscriber]

    def dispatch(self, event, message):
        print("Event", event, " - new message received", message)
        for subscriber, callback_fn in self.get_subscribers(event).items():
            callback_fn(message)
    
class TelegramSubscriber:
    def send_message(self, message):
        print("Telegram received a message: ", message)

class MailSubscriber:
    def send_mail(self, message):
        print("Email received a message: ", message)

class DataBaseSubscriber:
    def update(self, message):
        print("Database received a message: ", message)


class SlackSubscriber:
    def send_message(self, message):
        print("Slack received a message: ", message)

In [5]:
company = CompanyUpdate(events=['new entry'])

telegram = TelegramSubscriber()
email = MailSubscriber()
database = DataBaseSubscriber()

# subscribe all to the event "new entry"
company.register('new entry', telegram, telegram.send_message)
company.register('new entry', email, email.send_mail)
company.register('new entry', database)

# send a message
company.dispatch('new entry', message='Bob')

# the company stops using telegram and starts using slack
slack = SlackSubscriber()
company.unregister('new entry', telegram)
company.register('new entry', slack, slack.send_message)

company.dispatch('new entry', 'Alice')

Event new entry  - new message received Bob
Telegram received a message:  Bob
Email received a message:  Bob
Database received a message:  Bob
Event new entry  - new message received Alice
Email received a message:  Alice
Database received a message:  Alice
Slack received a message:  Alice


# Singleton design pattern

**Used for creating a single instance of an object across multiple parts of the code.**

Create a singleton that handles a logger (with an attribute `name`) and returns it every time the class is used. Use a metaclass for implementing it.
The method `log` should print the name of the logger (stored during the initialization of the class) and the message passed to the logger.

It should work with the following main.



In [6]:
class MetaSingleton(type):
    _instances = {}

    def __call__(cls, *args, **kwargs):
        if cls not in cls._instances:
            cls._instances[cls] = super(MetaSingleton, cls).__call__(*args, **kwargs)
        return cls._instances[cls]

class Logger(metaclass=MetaSingleton):
    def __init__(self, name):
        self.name = name
        print("Logger created: ", self.name)

    def log(self, message):
        print(f"{self.name}, message received: {message}")    




In [7]:
logger1 = Logger("logger 1")
logger2 = Logger("logger 2")

logger2.log("message to logger 2")

Logger created:  logger 1
logger 1, message received: message to logger 2


# Factory design pattern

**Used for separating creation from use.**

Define a class Country that creates a class for speaking the language and a class for paying a specified amount as a currency of the country. 
Create the countries Italy, England, and USA.

The script should work with the following main.

In [8]:
from abc import ABC, abstractmethod

class Country(ABC):
    @abstractmethod
    def get_language(self):
        pass

    @abstractmethod
    def get_currency(self):
        pass

class Language(ABC):
    @abstractmethod
    def speak(self):
        pass

class Currency(ABC):
    @abstractmethod
    def pay(self):
        pass

class Italian(Language):
    def speak(self):
        print("Io parlo Italiano.")

class Euro(Currency):
    def pay(self, amount):
        print(f"I payed {amount} euros.")

class English(Language):
    def speak(self):
        print("I speak English.")

class Pound(Currency):
    def pay(self, amount):
        print(f"I payed {amount} pounds.")

class Dollar(Currency):
    def pay(self, amount):
        print(f"I payed {amount} dollars.")

class Italy(Country):
    def get_language(self):
        return Italian()
    
    def get_currency(self):
        return Euro()

class England(Country):
    def get_language(self):
        return English()
    
    def get_currency(self):
        return Pound()

class USA(Country):
    def get_language(self):
        return English()
    
    def get_currency(self):
        return Dollar()

In [9]:
for factory in [Italy(), England(), USA()]:
    # retrieve the language and currency
    language = factory.get_language()
    currency = factory.get_currency()

    language.speak()
    currency.pay(20)


Io parlo Italiano.
I payed 20 euros.
I speak English.
I payed 20 pounds.
I speak English.
I payed 20 dollars.


# Miscellaneous exercises (these are really evil)

## Observer + Strategy

Use the observer pattern to model a system where your observers can subscribe to stocks in a stock market and make buy/sell decisions based on changes in the stock price. 

In particular, implement a strategy SellOnIncrease and a strategy BuyOnDecrease that the subscribers use for deciding their actions.
If the stock value changes and they decide to buy/sell, they remove/add the change amount from/to their wallet.

For example, if Jack (buy on decrease strategy) is subscribed to the Spotify stocks and the stock value decreases of 10 \$, Jack buys 10 \$ of that stock.


Here is the main.

In [10]:
class StockUpdate:
    def __init__(self, events):
        self.subscribers = {event: dict() for event in events}
    
    def get_subscribers(self, event):
        return self.subscribers.get(event, None)

    def register(self, event, subscriber, callback_fn=None):
        default_method = 'update'

        if callback_fn is None:
            try:
                callback_fn = getattr(subscriber, default_method)
            except:
                raise ValueError("Impossible to subscribe. The subscriber should provide "
                                 "an update method or pass a callback.")
        self.get_subscribers(event)[subscriber] = callback_fn
    
    def unregister(self, event, subscriber):
        subscribers_for_event = self.get_subscribers(event)
        if subscribers_for_event is not None and subscriber in subscribers_for_event:
            del self.get_subscribers(event)[subscriber]
    
    def dispatch(self, event, message):
        print(event, message)
        for subscriber, callback_fn in self.get_subscribers(event).items():
            callback_fn(message)

class StockSubscriber:
    def __init__(self, name, strategy):
        self.name = name
        self.strategy = strategy
        self.budget = 100

    def update(self, change):
        self.budget += self.strategy.action(change)
        print(self.name, "budget:", self.budget)

class StockStrategy(ABC):
    @abstractmethod
    def action(self, change):
        pass

class BuyOnDecreaseStrategy(StockStrategy):
    def action(self, change):
        if change < 0:
            return change
        else:
            return 0


class SellOnIncreaseStrategy(StockStrategy):
    def action(self, change):
        if change > 0:
            return change
        else:
            return 0

In [11]:

stock_market = StockUpdate(events=["spotify", "apple"])


jack = StockSubscriber("jack", BuyOnDecreaseStrategy())
nick = StockSubscriber("nick", BuyOnDecreaseStrategy())
kevin = StockSubscriber("kevin", SellOnIncreaseStrategy())
wendy = StockSubscriber("wendy", SellOnIncreaseStrategy())
alice = StockSubscriber("alice", SellOnIncreaseStrategy())

for investor in [jack, nick, kevin, wendy, alice]:
    stock_market.register("spotify", investor)

for investor in [jack, wendy, alice]:
    stock_market.register("apple", investor)

# price change
stock_market.dispatch("spotify", 30)
stock_market.dispatch("spotify", -10)
stock_market.dispatch("apple", -40)

stock_market.unregister("spotify", kevin)
stock_market.dispatch("spotify", -10)

spotify 30
jack budget: 100
nick budget: 100
kevin budget: 130
wendy budget: 130
alice budget: 130
spotify -10
jack budget: 90
nick budget: 90
kevin budget: 130
wendy budget: 130
alice budget: 130
apple -40
jack budget: 50
wendy budget: 130
alice budget: 130
spotify -10
jack budget: 40
nick budget: 80
wendy budget: 130
alice budget: 130


## State + Strategy

Draw and implement using the state pattern the following state machine `Traveler` for modeling flights between the four cities.

| | Cagliari | Rome | Milan| Paris|
|---| --- | ---| ---| ---|
|Cagliari| - | 30 €, 45 minutes | 40 €, 60 minutes | - | 
|Rome| 30 €, 45 minutes | - | 20 €, 30 minutes | 150 €, 100 minutes | 
|Milan| 40 €, 60 minutes | 20 €, 30 minutes | - | 50 €, 100 minutes | 
|Paris| - | 50 €, 100 minutes | 50 €, 100 minutes | - | 

Starting from the previous code, write the class planner, that given a list of routes e.g., `[["Cagliari", "Rome", "Paris"], ["Cagliari", "Milan", "Paris"]]` finds the cheapest route and the shortest (in minutes) route if the route is valid (i.e., if it ends in the desired destination). 



In [30]:
class Cities:
    """Utility class for storing the city names as strings."""
    Cagliari = "Cagliari"
    Rome = "Rome"
    Milan = "Milan"
    Paris = "Paris"

class CityState(ABC):

    def _travel(self, city, traveler):
        self._action(city, traveler)
        self._change_state(city, traveler)
        
    
    def _action(self, city, traveler):
        if city in self._destinations:
            print(f"Traveling to {city}")
            price, duration = self._destinations[city]
            traveler.price_payed += price
            traveler.time_traveled += duration
        else:
            print(f"Destination {city} is not available from here. Not traveling.")

    def _change_state(self, city, traveler):
        if city in self._destinations:
            traveler.set_state(states[city])
    
class Cagliari(CityState):
    _destinations = {
            Cities.Rome: [30, 45],
            Cities.Milan:  [40, 60],
            }
    name = Cities.Cagliari

class Rome(CityState):
    _destinations = {
        Cities.Cagliari: [30, 45],
        Cities.Milan: [20, 30],
        Cities.Paris: [150, 100],
    }
    name = Cities.Rome

class Milan(CityState):
    _destinations = {
        Cities.Cagliari: [40, 60],
        Cities.Rome: [20, 30],
        Cities.Paris: [50, 100]
    }
    name = Cities.Milan

class Paris(CityState):
    _destinations = {
        Cities.Rome: [50, 100],
        Cities.Milan: [50, 100]
    }
    name = Cities.Paris

states = {Cities.Cagliari: Cagliari(),
          Cities.Rome: Rome(),
          Cities.Milan: Milan(),
          Cities.Paris: Paris()
          }

class Traveler:
    def __init__(self, name):
        self.name = name
        self.state = Cagliari()
        self.price_payed = 0
        self.time_traveled = 0

    def travel(self, city):
        self.state._travel(city, self)
    
    def set_state(self, new_state):
        self.state = new_state


class Planner:
    def __init__(self, strategy):
        self.strategy = strategy

    def choose_route(self, routes):
        best_route = self.strategy.choose_route(routes)
        print(f"{self.strategy}, best route: {best_route}")

class Strategy(ABC):
    @staticmethod
    @abstractmethod
    def choose_route(routes):
        pass

class CheapStrategy(Strategy):
    def choose_route(routes):
        best_route = 0  # will store the index of the best route
        best_price = 20000  # will store the best price
        for i, route in enumerate(routes):
            t = Traveler("Cheap Traveler")
            for destination in route:
                t.travel(destination)
            
            if t.state.name == route[-1]:
                if t.price_payed < best_price:
                    best_price = t.price_payed
                    best_route = i
        return routes[best_route]

class FastStrategy(Strategy):
    def choose_route(routes):
        best_route = 0  # will store the index of the best route
        best_duration = 20000  # will store the lowest duration

        for i, route in enumerate(routes):
            t = Traveler("Fast Traveler")
            for destination in route:
                t.travel(destination)
            
            if t.state.name == route[-1]:
                if t.time_traveled < best_duration:
                    best_duration = t.time_traveled
                    best_route = i
        return routes[best_route]

In [31]:
# first part


t = Traveler("Tom")
t.travel(Cities.Paris)
t.travel(Cities.Rome)
t.travel(Cities.Milan)
t.travel(Cities.Paris)
t.travel(Cities.Cagliari)

# second part

print("\n\n")

route_1 = [Cities.Rome, Cities.Paris]
route_2 = [Cities.Milan, Cities.Paris]
route_3 = [Cities.Paris]
routes = [route_1, route_2, route_3]

cheap_planner = Planner(CheapStrategy)
cheap_planner.choose_route(routes)

fast_planner = Planner(FastStrategy)
fast_planner.choose_route(routes)


Destination Paris is not available from here. Not traveling.
Traveling to Rome
Traveling to Milan
Traveling to Paris
Destination Cagliari is not available from here. Not traveling.



Traveling to Rome
Traveling to Paris
Traveling to Milan
Traveling to Paris
Destination Paris is not available from here. Not traveling.
<class '__main__.CheapStrategy'>, best route: ['Milan', 'Paris']
Traveling to Rome
Traveling to Paris
Traveling to Milan
Traveling to Paris
Destination Paris is not available from here. Not traveling.
<class '__main__.FastStrategy'>, best route: ['Rome', 'Paris']


## Double dispatch (Visitor) with multiple visitors + (optional) Factory
Write a class Car that contains a list of parts (e.g., FrontLeftWheel, FrontRightWheel, RearLeftWheel, RearRightWheel, Engine). The parts are implemented with the double dispatch method. They have the attributes `name` and `price` and a method `accept` that takes a visitor as argument (the car also has a method `accept` that calls the `accept` method of the parts. Then implement two visitor classes, one that prints the part names, and one that prints the sum of the prices. 

Extra: you can implement different cars with factories that create different cars with different sets of parts (e.g., CheapWheels, FancyWheels, PowerfulEngine, CheapEngine etc.). Have fun 😈

In [34]:
class AbstractCarPart(ABC):
    def __init__(self, name, price):
        self.name = name
        self.price = price

    def accept(self, visitor):
        visitor.visit(self)

class Wheel(AbstractCarPart):
    pass

class Engine(AbstractCarPart):
    pass

class Seats(AbstractCarPart):
    pass

class Car:
    def __init__(self, name):
        self.name = name

        self._parts = [
            Wheel("front_left_wheel", 30),
            Wheel("front_right_wheel", 30),
            Wheel("rear_left_wheel", 30),
            Wheel("rear_right_wheel", 30),
            Engine("ferrari_engine", 1000),
            Seats("ferrari_seats", 200)
        ]
    
    def accept(self, visitor):
        for part in self._parts:
            part.accept(visitor)
    

class PrintPartsVisitor:
    @staticmethod
    def visit(part):
        print(f"Element: {part.name}")

class TotalPriceVisitor:
    total_price = 0

    def visit(self, part):
        self.total_price += part.price
        return self.total_price


In [35]:
car = Car("Ferrari")
# print out the part names using the PrintPartsVisitor
car.accept(PrintPartsVisitor())

# calculate the total price of the parts using the TotalPriceVisitor
total_visitor = TotalPriceVisitor()
car.accept(total_visitor)
print(f"Total Price = {total_visitor.total_price}")

Element: front_left_wheel
Element: front_right_wheel
Element: rear_left_wheel
Element: rear_right_wheel
Element: ferrari_engine
Element: ferrari_seats
Total Price = 1320
